In [ ]:
print(1)

In [7]:
# import plistlib
# import os
# p1 = "defaults_before.plist"
# print(os.path.exists(p1))
# with open(p1, "r") as f:
#     content = f.read()
#     # Remove any non-ASCII characters that could cause issues
#     content = ''.join(char for char in content if ord(char) < 128)
#     # Ensure content starts with proper plist format
#     if not content.strip().startswith('<?xml'):
#         content = '<?xml version="1.0" encoding="UTF-8"?>\n<!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">\n<plist version="1.0">\n' + content + '\n</plist>'
# with open(p1 + ".tmp", "w") as f:
#     f.write(content)
# d1 = plistlib.load(open(p1 + ".tmp", "rb"))

# p2 = "defaults_after.plist"
# with open(p2, "r") as f:
#     content = f.read()
#     content = ''.join(char for char in content if ord(char) < 128)
#     if not content.strip().startswith('<?xml'):
#         content = '<?xml version="1.0" encoding="UTF-8"?>\n<!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">\n<plist version="1.0">\n' + content + '\n</plist>'
# with open(p2 + ".tmp", "w") as f:
#     f.write(content)
# d2 = plistlib.load(open(p2 + ".tmp", "rb"))

# print(d1)
# print(d2)


True


ExpatError: not well-formed (invalid token): line 6656, column 35

In [13]:
import json
p1 = "temp/d1.json"
p2 = "temp/d2.json"
d1 = json.load(open(p1, "r"))
d2 = json.load(open(p2, "r"))



In [17]:
def deep_diff(d1, d2):
    diff = {}
    
    # Handle lists specially
    if isinstance(d1, list) and isinstance(d2, list):
        list_diff = {'added': [], 'removed': [], 'changed': []}
        
        # First, find exact matches and remove them from consideration
        remaining1 = d1.copy()
        remaining2 = d2.copy()
        
        # Remove exact matches
        for item1 in d1[:]:
            if item1 in d2:
                remaining1.remove(item1)
                remaining2.remove(item1)
        
        # For dictionaries in lists, try to match by common keys
        if remaining1 and remaining2 and all(isinstance(x, dict) for x in remaining1) and all(isinstance(x, dict) for x in remaining2):
            matched_pairs = []
            
            # Try to match items based on key intersection
            for i, item1 in enumerate(remaining1):
                best_match = None
                best_match_idx = None
                max_common_keys = 0
                
                for j, item2 in enumerate(remaining2):
                    if j not in [p[1] for p in matched_pairs]:
                        common_keys = set(item1.keys()) & set(item2.keys())
                        if len(common_keys) > max_common_keys:
                            max_common_keys = len(common_keys)
                            best_match = item2
                            best_match_idx = j
                
                if best_match and max_common_keys > 0:
                    matched_pairs.append((i, best_match_idx))
                    list_diff['changed'].append(deep_diff(item1, best_match))
            
            # Remove matched items from remaining lists
            for i1, i2 in reversed(matched_pairs):
                remaining1.pop(i1)
                remaining2.pop(i2)
        
        # Whatever is left is added or removed
        list_diff['added'] = remaining2
        list_diff['removed'] = remaining1
        
        return list_diff if list_diff['added'] or list_diff['removed'] or list_diff['changed'] else {}
    
    # Handle dictionaries
    elif isinstance(d1, dict) and isinstance(d2, dict):
        for k in d1:
            if k in d2:
                if isinstance(d1[k], (dict, list)) and isinstance(d2[k], (dict, list)):
                    nested_diff = deep_diff(d1[k], d2[k])
                    if nested_diff:
                        diff[k] = nested_diff
                elif d1[k] != d2[k]:
                    diff[k] = {'old': d1[k], 'new': d2[k]}
                    
        # Check for keys only in d2
        for k in d2:
            if k not in d1:
                diff[k] = {'old': None, 'new': d2[k]}
                
        # Check for keys only in d1
        for k in d1:
            if k not in d2:
                diff[k] = {'old': d1[k], 'new': None}
    
    # Handle primitive values
    else:
        if d1 != d2:
            return {'old': d1, 'new': d2}
    
    return diff

# Test the function
res = deep_diff(d1, d2)
# save to file
with open("temp/diff.json", "w") as f:
    json.dump(res, f, indent=2)

In [18]:
import plistlib

p1 = "temp/sw.plist"
p2 = "temp/dock.plist"
d1 = plistlib.load(open(p1, "rb"))
d2 = plistlib.load(open(p2, "rb"))

# res = deep_diff(d1, d2)
# with open("temp/dock_diff.json", "w") as f:
#     json.dump(res, f, indent=2)


In [21]:
d2

{'tilesize': 67.0,
 'autohide': True,
 'version': 1,
 'last-analytics-stamp': [752597068.93466],
 'persistent-others': [{'GUID': 4152855558,
   'tile-data': {'showas': 1,
    'parent-mod-date': 152595411841351,
    'file-type': 2,
    'is-beta': False,
    'book': b'book|\x02\x00\x00\x00\x00\x04\x100\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00x\x01\x00\x00\x04\x00\x00\x00\x03\x03\x00\x00\x00\x00\x00 \x05\x00\x00\x00\x01\x01\x00\x00Users\x00\x00\x00\x04\x00\x00\x00\x01\x01\x00\x00petr\t\x00\x00\x00\x01\x01\x00\x00Downloads\x00\x00\x00\x0c\x00\x00\x00\x01\x06\x00\x00\x10\x00\x00\x00 \x00\x00\x00,\x00\x00\x00\x08\x00\x00\x00\x04\x03\x00\x00\xdeA\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x04\x03\x00\x00\xbdn\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x04\x03\x00\x00\xf0n\x00\x00\x00\x00\x00\x00\x0c\x00\x00\x00\x01\x06\x00\x00T\x00\x00\x00d\x00\x00\x00t\x00\x00\x00\x08\x00\x00\x00\x00\x04\x00\x00A\xc6